In [1]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint


In [67]:
# load data

ports = pd.read_csv('data/ports_10.csv')
dm = pd.read_csv('data/distance_matrix.csv')
fleet = pd.read_csv('data/fleet.csv')

In [59]:
def create_contracts(seed):
    """
    A function that creates 4 random cargo contracts at each env step.\ 
    There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
    The function :  
    * returns a 1d array of 50 elements
    * all array elements are integers from 0 to 4
    * the sum of the array must be 4
    """
    import numpy as np
    number_of_contracts = 4
    np.random.seed(seed)
    x = np.random.multinomial( number_of_contracts, [1/5]*5, None)
    z = np.zeros(shape=45,dtype=np.int32)
    c = np.concatenate((x,z))
    np.random.shuffle(c)
    return c 




In [66]:
def cii_required(dwt, year=2023):
    """
    a function calculating the annual required cii of a vessel given a year
    """
    a = 4745
    c = 0.622
    z = {2023:5, 2024:7, 2025:9, 2026:8}
    cii_ref = a * dwt ** (-c)
    cii_required = ((100 - z[year])/100) * cii_ref
    return cii_required



In [68]:
fleet

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed
0,handy_1,handymax,32000,0,0,somewhere,10
1,handy_2,handymax,32000,0,0,somewhere,10
2,supra_1,supramax,52000,0,0,somewhere,12
3,supra_2,supramax,52000,0,0,somewhere,12
4,ultra_1,ultramax,62000,0,0,somewhere,14
5,ultra_2,ultramax,62000,0,0,somewhere,14
6,pana_1,panamax,75000,0,0,somewhere,12
7,pana_2,panamax,75000,0,0,somewhere,12
8,kamsar_1,kamsarmax,82000,0,0,somewhere,14
9,kamsar_2,kamsarmax,82000,0,0,somewhere,14


In [ ]:
def cii_attained(dwt, distance_sailed, co2_emissions):
    """
    a function calculating the annual attained cii of a vessel
    """
    cii_attained = (co2_emissions * 1_000_000)/(dwt*distance_sailed)
    return cii_attained

In [ ]:
def cii_rating(attained_cii, required_cii):
    """
    a function calculating the cii rating of a vessel
    """
    d1, d2, d3, d4 = 0.86, 0.94, 1.06, 1.18
    rating = attained_cii / required_cii
    if rating <= d1 :
        return "Rating A"
    elif rating <= d2 :
        return "Rating B"
    elif rating <= d3 :
        return "Rating C"
    elif rating <= d4 :
        return "Rating D"
    else:
        return "Rating E"


In [ ]:
def can_reach(vessel_location, vessel_set_of_speeds, start_port, start_time, free, previous_end_port):
    """
    function checking if a vessel can reach a start port on time
    """
    # if the vessel is free
    # the vessel has to reach the new start port on time
    # u = Dx / Dt 


    # # if the vessel is not free 
    # the vessel has to reach the previous end port on time # use set of speeds 

    # u = Dx / Dt 
    # and then
    # the vessel has to reach the new start port on time


    pass

In [ ]:
def can_serve(vessels_df, contract_df, ports_df):
    """
    function checking if a vessel meets the conditions to serve a contract 
    """
    # # Check if vessel has the capacity to transport the cargo
    # vessel_capacity >= cargo_volume
    
    #     # If it does then check if it is free or already serving a cargo
    #     vessel_free == Yes:

    #         # Check if the vessel can reach the start port at start_time
    #         can_reach()    





    pass

In [3]:
import pandas as pd

fleet = pd.read_csv('data/fleet.csv')

In [4]:
fleet

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed
0,handy_1,handymax,40000,0,0,somewhere,0
1,handy_2,handymax,40000,0,0,somewhere,0
2,supra_1,supramax,50000,0,0,somewhere,0
3,supra_2,supramax,50000,0,0,somewhere,0
4,ultra_1,ultramax,60000,0,0,somewhere,0
5,ultra_2,ultramax,60000,0,0,somewhere,0
6,pana_1,panamax,70000,0,0,somewhere,0
7,pana_2,panamax,70000,0,0,somewhere,0
8,kamsar_1,kamsarmax,80000,0,0,somewhere,0
9,kamsar_2,kamsarmax,80000,0,0,somewhere,0


In [2]:
data_dict = {
    "fleet_path":'data/fleet.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}

In [ ]:
# set fleet at random ports

In [42]:
import gym 
import pandas as pd
import numpy as np
from gym import spaces

class CarbonEnv(gym.Env):
    """
    A custom environment for the Carbon emission problem.
    Follows the openai gym logic 
    """
    
    def __init__(self,data_dict):
        super().__init__()
       


        self.fleet = pd.read_csv(data_dict['fleet_path'])
        self.ports = pd.read_csv(data_dict['ports_path'])
        self.dm = pd.read_csv(data_dict['dm_path'])
        self.fleet['cii_required'] = self.fleet['dwt'].map(cii_required)

        # todo set fleet at random ports
        # todo set fleet speeds [0, 10, 12, 14] knots
        # maybe better [10,12,14] and calculate cii only if ship == selected



        self.observation_space = spaces.Dict({
            "contracts" : spaces.Box(low=0,high=4,shape=(50,), dtype=np.int32),
            "ships" : spaces.Discrete(10,),
            "speeds" : spaces.Box(low=0,high=2,shape=(10,), dtype=np.int32)
            })


        self.action_space = spaces.Dict({
            "choose_contract" : spaces.Discrete(4,), # choose 1,2,3 or 4 contracts
            "choose_ship" : spaces.Discrete(2,), # which of the 2 ships in the class to pick
            "choose_speed" : spaces.Discrete(3,) # which of the 3 speeds to pick

        })

    def set_obs_space(self):
        self.observation_space['contracts'] = self.create_contracts()
        

    def set_act_space(self,):
        pass


    def step(self, action):
        # Execute one time step in the environment
        pass

    def reset(self):
        # Reset the state of the environment to an initial state
        obs = self.create_contracts()
        return obs
        


    def render(self, mode='human', close=False):
        # Render the environment to the screen
        pass


    def create_contracts(self, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c

    def cii_required(dwt, year=2023):
        """
        a method to calculate the required cii of a fleet given a year
        """
        a = 4745
        c = 0.622
        z = {2023: 5, 2024: 7, 2025: 8, 2026: 9}
        cii_ref = a * dwt ** (-c)
        cii_req = ((100-z[year]) / 100 ) * cii_ref
        return cii_req


In [35]:
envo = CarbonEnv(data_dict)

In [37]:
envo.observation_space['contracts']

Box(0, 4, (50,), int32)

In [41]:
envo.reset()

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0])